In [82]:
from rdkit.Chem import AllChem
from openff.interchange import Interchange
from openff.toolkit import Molecule
from openmm.app import ForceField, HBonds, PME, Simulation

from openmm.unit import kelvin, femtosecond, picosecond,  kilojoules, mole, nanometer, dalton
from openmm import app
import openmm
from rdkit import Chem
from rdkit.Chem import AllChem

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import typing as T

# import h5py
import numpy as np
import mdtraj as md

smiles = 'O=C(CCCn1c(=O)c2ccccc2n(CC(=O)Nc2cccc(Cl)c2)c1=O)NCc1ccc2c(c1)OCO2'
smiles = 'CCCCO'

In [94]:
Tmin = 100
Tmax = 310
n = 2
for i in range(n):
    print(Tmin + (i)*(Tmax-Tmin)/(n-1))

ZeroDivisionError: division by zero

In [71]:

mol = Molecule.from_smiles(smiles)
mol.generate_conformers()

In [72]:
top = mol.to_topology().to_openmm()
pos = mol.conformers[0].to_openmm()

In [73]:
model = app.Modeller(top, pos)

In [76]:
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
smirnoff = SMIRNOFFTemplateGenerator(molecules=mol, forcefield="openff-2.1.0.offxml")
forcefield = ForceField('amber/tip3p_standard.xml')
forcefield.registerTemplateGenerator(smirnoff.generator)


In [77]:
model.addSolvent(forcefield, padding=1*nanometer)


In [80]:
system = forcefield.createSystem(model.topology, 
                        nonbondedMethod=PME,
                        nonbondedCutoff=1*nanometer, 
                        constraints=HBonds, 
                        rigidWater=True, 
                        )

In [87]:
system.

AttributeError: 'System' object has no attribute 'topology'

In [81]:
integrator = openmm.LangevinMiddleIntegrator(300*kelvin, 
                                             1/(2*picosecond), 
                                             1*femtosecond)

In [88]:
?openmm.LangevinMiddleIntegrator

Init signature: openmm.LangevinMiddleIntegrator(*args)
Docstring:     
This is an Integrator which simulates a System using Langevin dynamics, with the LFMiddle discretization (J. Phys. Chem. A 2019, 123, 28, 6056-6079). This method tend to produce more accurate configurational sampling than other discretizations, such as the one used in LangevinIntegrator.


The algorithm is closely related to the BAOAB discretization (Proc. R. Soc. A. 472: 20160138). Both methods produce identical trajectories, but LFMiddle returns half step (leapfrog) velocities, while BAOAB returns on-step velocities. The former provide a much more accurate sampling of the thermal ensemble.
Init docstring:
__init__(self, temperature, frictionCoeff, stepSize) -> LangevinMiddleIntegrator
__init__(self, other) -> LangevinMiddleIntegrator
Create a LangevinMiddleIntegrator.

Parameters
----------
temperature : double
    the temperature of the heat bath (in Kelvin)
frictionCoeff : double
    the friction coefficient whi

In [90]:
# simulation = Simulation(model.topology, system, integrator)
?Simulation

Init signature:
Simulation(
    topology,
    system,
    integrator,
    platform=None,
    platformProperties=None,
    state=None,
)
Docstring:     
Simulation provides a simplified API for running simulations with OpenMM and reporting results.

A Simulation ties together various objects used for running a simulation: a Topology, System,
Integrator, and Context.  To use it, you provide the Topology, System, and Integrator, and it
creates the Context automatically.

Simulation also maintains a list of "reporter" objects that record or analyze data as the simulation
runs, such as writing coordinates to files or displaying structures on the screen.  For example,
the following line will cause a file called "output.pdb" to be created, and a structure written to
it every 1000 time steps:

simulation.reporters.append(PDBReporter('output.pdb', 1000))
Init docstring:
Create a Simulation.

Parameters
----------
topology : Topology
    A Topology describing the the system to simulate
system : 

In [89]:
[x for x in dir(system) if x[0] != '_']

['addConstraint',
 'addForce',
 'addParticle',
 'getConstraintParameters',
 'getDefaultPeriodicBoxVectors',
 'getForce',
 'getForces',
 'getNumConstraints',
 'getNumForces',
 'getNumParticles',
 'getParticleMass',
 'getVirtualSite',
 'isVirtualSite',
 'removeConstraint',
 'removeForce',
 'setConstraintParameters',
 'setDefaultPeriodicBoxVectors',
 'setParticleMass',
 'setVirtualSite',
 'this',
 'thisown',
 'usesPeriodicBoundaryConditions']